# Code to get ground truth prediction results

### External validation

In [4]:
'''
ground truth for cross-dataset predictinos
'''

import pandas as pd
import numpy as np
import scipy.io as sio
import os
import time
import random
from tqdm.auto import tqdm
from scipy import stats
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV, cross_val_score, KFold
from sklearn.feature_selection import SelectPercentile, f_regression, r_regression
from sklearn.pipeline import Pipeline
from sklearn.metrics import make_scorer
import argparse
import mat73


def calc_q2(true_vals, pred_vals):
    return 1 - ((true_vals-pred_vals)**2).mean() / ((true_vals-true_vals.mean())**2).mean()
 
def r_to_p(r, n, tails=2):
    t = r / np.sqrt((1-r**2)/ (n-2) )
    
    if tails==2:
        p = 2*stats.t.sf(abs(t), df=n-2)
    elif tails==1:  # for one-sided positive test only
        p = stats.t.sf(t, df=n-2)   # positive only
    return p

# custom scorer (Pearson's r) for grid search
def my_custom_loss_func(y_true, y_pred):
    return np.corrcoef(y_true, y_pred)[0, 1]


# Set running parameters
per_feat = 0.01
k = 5  # for inner grid search
score = make_scorer(my_custom_loss_func, greater_is_better=True)

# set save and load paths
data_load_path = '/home/mjr239/project/repro_data/'  # /home/mjr239/project/repro_data/

# load in external dataset
df_dataset_pheno_sample_size = pd.read_csv(os.path.join(data_load_path, 'pheno_dataset_sample_size.csv'))

# make list for future dataframe
pheno_gt = []
train_dataset_gt = []
test_dataset_gt = []
num_train_gt = []
num_test_gt = []
rval_gt = []
pval_gt = []

# loop over all possible phenotypes
for pheno in ['mr_scaled', 'mr', 'age', 'attn']:
    
    # get relevant datasets for each phenotype
    if pheno=='age':
        all_datasets = ['hbn', 'hcpd', 'pnc']
    elif pheno=='mr_scaled':  # no age/abcd
        all_datasets = ['abcd', 'hbn', 'hcpd']
    else:
        all_datasets = ['abcd', 'hbn', 'hcpd', 'pnc']
    
    ######################## Load in all datasets for this phenotype ########################
    datasets = dict()
    for dataset_name in all_datasets:   
        
        # load in data
        try:
            dat = sio.loadmat( os.path.join(data_load_path, dataset_name+'_feat.mat') )
        except NotImplementedError:  # if matlab file not compatible (v7.3), need to use hdf5 reader
            dat = mat73.loadmat(os.path.join(data_load_path, dataset_name+'_feat.mat'))

        # read in phenotype and covariates
        df_tmp = pd.read_csv( os.path.join(data_load_path, dataset_name+'_python_table.csv') )
        all_possible_covars = ['age', 'sex', 'motion_vals', 'site', 'family_id']
        df_include_vars = [c for c in all_possible_covars if ((c in df_tmp.keys()) & (pheno!=c))]  # get df of covariates
        df_include_vars.append(pheno)
        df_tmp = df_tmp[df_include_vars]
        good_idx = np.where(df_tmp.isna().sum(axis=1)==0)[0]  # remove rows with missing data
        df_tmp = df_tmp.iloc[good_idx, :]
        df_tmp['sex'] = df_tmp.sex.replace('F', 0).replace('M', 1)  # replace sex variables

        # add this to dictionary of datasets
        datasets[dataset_name] = dict()
        datasets[dataset_name]['X'] = dat['X'][:, good_idx]   
        datasets[dataset_name]['behav'] = df_tmp    
        datasets[dataset_name]['n'] = len(datasets[dataset_name]['behav'])

        # add in covariate array to dictionary
        all_keys = datasets[dataset_name]['behav'].keys()
        covar_keys = [k for k in all_keys if ((k!=pheno) and (k!='site') and (k!='family_id'))]  # motion, sex, age
        datasets[dataset_name]['C_all'] = np.array(datasets[dataset_name]['behav'][covar_keys])

    ######################## Train models ########################
    for train_dataset_name in all_datasets:
           
        # covariate regression
        Beta = np.matmul( np.matmul( np.linalg.inv( np.matmul(datasets[train_dataset_name]['C_all'].T,
                                                              datasets[train_dataset_name]['C_all'])),
                                    datasets[train_dataset_name]['C_all'].T),
                        datasets[train_dataset_name]['X'].T)
        X_train = datasets[train_dataset_name]['X'].T - np.matmul(datasets[train_dataset_name]['C_all'], Beta)
        y_train = list(datasets[train_dataset_name]['behav'][pheno])
        print(X_train.shape)
    
        # feature selection
        r = r_regression(X_train, y_train)
        p = r_to_p(r, len(y_train)) 
        pthresh = np.percentile(p, 100*per_feat)
        sig_feat_loc = np.where(p<pthresh)[0]
    
        # model fitting
        # for kfold_seed_idx, kfold_seed in enumerate(range(num_kfold_repeats)):
        kfold_seed = 0
        inner_cv = KFold(n_splits=k, shuffle=True, random_state=kfold_seed) 
        regr = GridSearchCV(estimator=Ridge(), param_grid={'alpha':np.logspace(-3, 3, 7)}, cv=inner_cv, scoring=score)
        regr.fit(X_train[:, sig_feat_loc], y_train)        
        
        ######################## Test models ########################
        test_datasets = [d for d in all_datasets if d!=train_dataset_name]
        for test_dataset_name in test_datasets:
            # evaluate in all data (not yet resampled) using saved coefficients for model/covariates
            X_all_eval = np.copy(datasets[test_dataset_name]['X'].T)
            y_all_eval = np.squeeze(np.array(datasets[test_dataset_name]['behav'][pheno]))
            C_all_eval = datasets[test_dataset_name]['C_all']
            X_all_eval_corrected = X_all_eval[:, sig_feat_loc] - np.matmul(C_all_eval, Beta[:, sig_feat_loc])
            yp_all_eval = regr.predict(X_all_eval_corrected)
            

            # print external validation results
            print('***************************************************')
            print('Pheno: {:s}, training dataset: {:s} (n={:d}) '\
                    'test dataset: {:s} (n={:d})'.format(pheno, train_dataset_name,
                                                        len(y_train), test_dataset_name, len(y_all_eval)))          
            rval = np.corrcoef(y_all_eval, yp_all_eval)[0, 1]
            print('r={:.5f} (p={:.5f})'.format(rval, r_to_p(rval, len(yp_all_eval), tails=1)))
            
            # make list for future dataframe
            pheno_gt.append(pheno)
            train_dataset_gt.append(train_dataset_name)
            test_dataset_gt.append(test_dataset_name)
            num_train_gt.append(len(y_train))
            num_test_gt.append(len(y_all_eval))
            rval_gt.append(rval)
            pval_gt.append( r_to_p(rval, len(yp_all_eval), tails=1) )

# save
df_gt = pd.DataFrame({'pheno':pheno_gt, 'train_dataset':train_dataset_gt,
                     'test_dataset':test_dataset_gt, 'num_train':num_train_gt,
                     'num_test':num_test_gt, 'rval':rval_gt,
                     'pval':pval_gt})
df_gt.to_csv('ground_truth_update.csv', index=False)

abcd
hbn
hcpd
(7822, 35778)
***************************************************
Pheno: mr_scaled, training dataset: abcd (n=7822) test dataset: hbn (n=1024)
r=0.08237 (p=0.00418)
***************************************************
Pheno: mr_scaled, training dataset: abcd (n=7822) test dataset: hcpd (n=424)
r=0.23255 (p=0.00000)
(1024, 35778)
***************************************************
Pheno: mr_scaled, training dataset: hbn (n=1024) test dataset: abcd (n=7822)
r=0.06991 (p=0.00000)
***************************************************
Pheno: mr_scaled, training dataset: hbn (n=1024) test dataset: hcpd (n=424)
r=0.23154 (p=0.00000)
(424, 35778)
***************************************************
Pheno: mr_scaled, training dataset: hcpd (n=424) test dataset: abcd (n=7822)
r=0.08815 (p=0.00000)
***************************************************
Pheno: mr_scaled, training dataset: hcpd (n=424) test dataset: hbn (n=1024)
r=0.11405 (p=0.00013)
abcd
hbn
hcpd
pnc
(7822, 35778)
*********